In [ ]:
import numpy as np
import pandas as pd 
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import datasets, models, transforms

torch.manual_seed(42)
np.random.seed(42)

In [ ]:
from torch.hub import load_state_dict_from_url
__all__ = ['ResNet', 'resnet18']
model_urls = {
    'resnet18': 'https://download.pytorch.org/models/resnet18-5c106cde.pth',}

def conv3x3(in_planes, out_planes, stride=1, groups=1, dilation=1):
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=dilation, groups=groups, bias=False, dilation=dilation)
def conv1x1(in_planes, out_planes, stride=1):
    return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)
class BasicBlock(nn.Module):
    expansion = 1
    def __init__(self, inplanes, planes, stride=1, downsample=None, groups=1,
                 base_width=64, dilation=1, norm_layer=None):
        super(BasicBlock, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        if groups != 1 or base_width != 64:
            raise ValueError('BasicBlock only supports groups=1 and base_width=64')
        if dilation > 1:
            raise NotImplementedError("Dilation > 1 not supported in BasicBlock")
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = norm_layer(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = norm_layer(planes)
        self.downsample = downsample
        self.stride = stride
    def forward(self, x):
        identity = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        if self.downsample is not None:
            identity = self.downsample(x)
        out += identity
        out = self.relu(out)
        return out
class Bottleneck(nn.Module):
    expansion = 4
    def __init__(self, inplanes, planes, stride=1, downsample=None, groups=1,
                 base_width=64, dilation=1, norm_layer=None):
        super(Bottleneck, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        width = int(planes * (base_width / 64.)) * groups
        self.conv1 = conv1x1(inplanes, width)
        self.bn1 = norm_layer(width)
        self.conv2 = conv3x3(width, width, stride, groups, dilation)
        self.bn2 = norm_layer(width)
        self.conv3 = conv1x1(width, planes * self.expansion)
        self.bn3 = norm_layer(planes * self.expansion)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride
    def forward(self, x):
        identity = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)
        out = self.conv3(out)
        out = self.bn3(out)
        if self.downsample is not None:
            identity = self.downsample(x)
        out += identity
        out = self.relu(out)
        return out
class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes=1000, zero_init_residual=False,
                 groups=1, width_per_group=64, replace_stride_with_dilation=None,
                 norm_layer=None):
        super(ResNet, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        self._norm_layer = norm_layer

        self.inplanes = 64
        self.dilation = 1
        if replace_stride_with_dilation is None:
            replace_stride_with_dilation = [False, False, False]
        if len(replace_stride_with_dilation) != 3:
            raise ValueError("replace_stride_with_dilation should be None "
                             "or a 3-element tuple, got {}".format(replace_stride_with_dilation))
        self.groups = groups
        self.base_width = width_per_group
        self.conv1 = nn.Conv2d(3, self.inplanes, kernel_size=7, stride=2, padding=3,
                               bias=False)
        self.bn1 = norm_layer(self.inplanes)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2,
                                       dilate=replace_stride_with_dilation[0])
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2,
                                       dilate=replace_stride_with_dilation[1])
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2,
                                       dilate=replace_stride_with_dilation[2])
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, (nn.BatchNorm2d, nn.GroupNorm)):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
        if zero_init_residual:
            for m in self.modules():
                if isinstance(m, Bottleneck):
                    nn.init.constant_(m.bn3.weight, 0)
                elif isinstance(m, BasicBlock):
                    nn.init.constant_(m.bn2.weight, 0)

    def _make_layer(self, block, planes, blocks, stride=1, dilate=False):
        norm_layer = self._norm_layer
        downsample = None
        previous_dilation = self.dilation
        if dilate:
            self.dilation *= stride
            stride = 1
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                conv1x1(self.inplanes, planes * block.expansion, stride),
                norm_layer(planes * block.expansion),
            )
        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample, self.groups,
                            self.base_width, previous_dilation, norm_layer))
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes, groups=self.groups,
                                base_width=self.base_width, dilation=self.dilation,
                                norm_layer=norm_layer))

        return nn.Sequential(*layers)

    def _forward_impl(self, x):
        # See note [TorchScript super()]
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        x = nn.functional.softmax(x, 1)

        return x

    def forward(self, x):
        return self._forward_impl(x)


def _resnet(arch, block, layers, pretrained, progress, **kwargs):
    model = ResNet(block, layers, **kwargs)
    if pretrained:
        state_dict = load_state_dict_from_url(model_urls[arch],
                                              progress=progress)
        model.load_state_dict(state_dict)
    return model
def resnet18(pretrained=False, progress=True, **kwargs):
    return _resnet('resnet18', BasicBlock, [2, 2, 2, 2], pretrained, progress,
                   **kwargs)

In [ ]:
data_dir = ""
TEST = ''
TRAIN = ''
VAL =''

def data_transforms(phase):
    if phase == TRAIN:
        transform = transforms.Compose([
            transforms.Resize((50, 50)),
            transforms.ToTensor(),
#             transforms.Normalize([1.2201, 0.4040, 0.7327], [0.3852, 0.5240, 0.5857]),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        ])
        
    if phase == VAL:
        transform = transforms.Compose([
            transforms.Resize((50, 50)),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        ])
    
    if phase == TEST:
        transform = transforms.Compose([
            transforms.Resize((50, 50)),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        ])        
        
    return transform
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms(x)) 
                  for x in [TRAIN, VAL, TEST]}
dataloaders = {TRAIN: torch.utils.data.DataLoader(image_datasets[TRAIN], batch_size = 64, shuffle=True), 
               VAL: torch.utils.data.DataLoader(image_datasets[VAL], batch_size = 256, shuffle=True), 
               TEST: torch.utils.data.DataLoader(image_datasets[TEST], batch_size = 256, shuffle=True)}
dataset_sizes = {x: len(image_datasets[x]) for x in [TRAIN, VAL, TEST]}
classes = image_datasets[TRAIN].classes

In [ ]:
classes, counts = np.unique(dataloaders[TRAIN].dataset.targets, return_counts=True)
print(classes)
print(counts)
class_inverse_priors = {c: 1 / (count/len(dataloaders[TRAIN].dataset.targets)) for c, count in zip(list(classes), counts)}
print(class_inverse_priors)
cost_matrix = pd.DataFrame(data = [[0, class_inverse_priors[0]], [5 * class_inverse_priors[1], 0]], index=['actual negative (0)', 'actual positive (1)'], columns=['predict negative (0)', 'predict positive (1)'])
print(cost_matrix)

In [ ]:
def get_cost(preds, labels, cost_matrix):
    preds = preds.detach().cpu().numpy()
    labels = labels.detach().cpu().numpy()
    
    cost = []
    for y_hat, y_actual in zip(preds, labels):
        cost.append(get_cost_from_matrix(y_hat=y_hat, \
                                         y_actual=y_actual,\
                                         cost_matrix=cost_matrix))
        
    return sum(cost)

def get_cost_from_matrix(y_hat, y_actual, cost_matrix):    
    if y_actual == y_hat == 0:
        return cost_matrix.loc['actual negative (0)', 'predict negative (0)']

    if y_actual == y_hat == 1:
        return cost_matrix.loc['actual positive (1)', 'predict positive (1)']

    if (y_actual == 1) & (y_hat == 0):
        return cost_matrix.loc['actual positive (1)', 'predict negative (0)']

    if (y_actual == 0) & (y_hat == 1):
        return cost_matrix.loc['actual negative (0)', 'predict positive (1)']

In [ ]:
def weighted_cross_entropy(output, target, cost_matrix):
    
    output = torch.clamp(output,min=1e-5,max=1-1e-5)
    
    c10 = cost_matrix.loc['actual positive (1)', 'predict negative (0)']
    c01 = cost_matrix.loc['actual negative (0)', 'predict positive (1)']
    
    loss = c10 * (target * torch.log(output[:, 1])) +\
    c01 * ((1 - target) * torch.log(1 - output[:, 1]))
               
    return torch.neg(torch.mean(loss))

In [ ]:
def train(model, dataloader, optimizer, criterion, epoch, num_epochs):
    
    print("Epoch: {}/{}".format(epoch + 1, num_epochs))
    print("=" * 10)
    
    model.train()

    running_loss = 0.0
    running_corrects = 0
    total_cost = 0.0
    
    for data in dataloader:
        
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        cost = get_cost(preds, labels, cost_matrix)        
        loss = criterion(outputs, labels, cost_matrix)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)
        total_cost += cost
        
    epoch_loss = running_loss / dataset_sizes[TRAIN]
    epoch_acc = running_corrects.double() / dataset_sizes[TRAIN]
        
    print('Train:  {:.4f}: , Accuracy: {:.4f},\
    Cost: {:.4f}'.format(epoch_loss, epoch_acc, total_cost))
    
    return epoch_loss, epoch_acc, total_cost

In [ ]:
def evaluate(model, dataloader, criterion, phase, epoch, save=True):
        
    model.eval()
    running_loss = 0.0
    running_corrects = 0
    total_cost = 0.0
    
    with torch.no_grad():
        for data in dataloaders[phase]:
        
            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            
            cost = get_cost(preds, labels, cost_matrix)
            loss = criterion(outputs, labels, cost_matrix)
            
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)
            total_cost += cost
            
    epoch_loss = running_loss / dataset_sizes[phase]
    epoch_acc = running_corrects.double() / dataset_sizes[phase]
    
    print('{}:  {:.4f}: , Accuracy: {:.4f},\
    Cost: {:.4f}'.format(phase, epoch_loss, epoch_acc, total_cost))
    
    if save:
      global best_valid_cost
      if total_cost < best_valid_cost:
        best_valid_cost = total_cost
        print('Saving..')
        state = {
            'net': model.state_dict(),
            'epoch': epoch,
            'acc': epoch_acc,
            'cost': total_cost,
        }
        if not os.path.isdir('checkpoint'):
          os.mkdir('checkpoint')
        torch.save(state, './checkpoint/ckpt.pth')

    
    return epoch_loss, epoch_acc, total_cost

In [ ]:
resnet = resnet18(pretrained=True)
resnet.fc = nn.Linear(512, 2)
resnet = resnet.to(device)

criterion = weighted_cross_entropy
optimizer = optim.SGD(resnet.parameters(), lr=0.01, weight_decay=0.05)

In [ ]:
num_epochs = 15

best_valid_cost = 1e6

train_loss, valid_loss, train_acc, valid_acc,train_cost, valid_cost = [], [], [], [], [], []


for epoch in range(num_epochs):
    tr_epoch_loss, tr_epoch_acc, tr_epoch_cost= train(resnet, 
                                                       dataloaders[TRAIN], 
                                                       optimizer,
                                                       criterion,
                                                       epoch,
                                                       num_epochs)

    train_loss.append(tr_epoch_loss)
    train_acc.append(tr_epoch_acc)
    train_cost.append(tr_epoch_cost)

    val_epoch_loss, val_epoch_acc, val_epoch_cost = evaluate(resnet, 
                                                              dataloaders,
                                                              criterion,
                                                              VAL,
                                                              epoch,
                                                              save=True)

    valid_loss.append(val_epoch_loss)
    valid_acc.append(val_epoch_acc)
    valid_cost.append(val_epoch_cost)

In [ ]:
checkpoint = torch.load('./checkpoint/ckpt.pth')
resnet.load_state_dict(checkpoint['net'])
best_cost = checkpoint['cost']
current_epoch = checkpoint['epoch']

print(best_cost)
print(current_epoch)

In [ ]:
evaluate(resnet, dataloaders, criterion, TEST, epoch, save=False)

In [ ]:
results = pd.DataFrame({'train loss': train_loss,
              'train acc': [x.item() for x in train_acc],
              'train cost': train_cost, 
              'valid loss': valid_loss,
              'valid acc': [x.item() for x in valid_acc],
              'valid cost': valid_cost})